### Log Reader  

In [1]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="gemma3:1b", base_url="http://localhost:11434")

### Read the log file

In [2]:
import os
from langchain.tools import tool
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

@tool
def summarize_log() -> str:
    """Read and return summary of the first few lines from each .log file. DON'T get into details of them."""
    log_dir = './logs' 
    db_path = "./log_db"
    all_logs = []

    for file in os.listdir(log_dir):
        with open(os.path.join(log_dir, file)) as f:
            all_logs.extend([f"{file}: {line.strip()}" for line in f.readlines()[:50]]) 
    summary = "\n".join(all_logs) or "No log files found"
    
    # Check if log directory exists
    if not os.path.exists(db_path) and os.path.isdir(db_path):
        # splitting
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = splitter.create_documents([summary])    
        
        # embedding
        embedding = OllamaEmbeddings(model="nomic-embed-text")
        db = Chroma.from_documents(chunks, embedding, persist_directory=db_path)
        db.persist()
        summary += "\n\n[Embedding] DB already exists"
    else:
        summary += "\n\n[Embedding] No DB found (skipped creation)"
    
    return summary

In [3]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool

# Define tools
@tool
def add_number(a: int, b: int) -> int:
    """Add two numbers and return the result"""
    return int(a) + int(b)

@tool
def substract_number(a: int, b: int) -> int:
    """Subtract two numbers and return the result"""
    return int(a) - int(b)

@tool
def multiply_number(a: int, b: int) -> int:
    """Multiply two numbers and return the result"""
    return int(a) * int(b)

# Register tools
tools = [add_number, substract_number, multiply_number, summarize_log] 

# Create agent (use structured chat to support multi-input tools)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

C:\Users\Korisnik\AppData\Local\Temp\ipykernel_3232\3060087492.py:24: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [4]:
tool_by_name = {tool.name: tool for tool in tools}
tool_by_name

{'add_number': StructuredTool(name='add_number', description='Add two numbers and return the result', args_schema=<class 'langchain_core.utils.pydantic.add_number'>, func=<function add_number at 0x000001CCF2C92CA0>),
 'substract_number': StructuredTool(name='substract_number', description='Subtract two numbers and return the result', args_schema=<class 'langchain_core.utils.pydantic.substract_number'>, func=<function substract_number at 0x000001CCFEDE4FE0>),
 'multiply_number': StructuredTool(name='multiply_number', description='Multiply two numbers and return the result', args_schema=<class 'langchain_core.utils.pydantic.multiply_number'>, func=<function multiply_number at 0x000001CCFF4EEC00>),
 'summarize_log': StructuredTool(name='summarize_log', description="Read and return summary of the first few lines from each .log file. DON'T get into details of them.", args_schema=<class 'langchain_core.utils.pydantic.summarize_log'>, func=<function summarize_log at 0x000001CCFA97A660>)}

In [ ]:
#query = "What is the sum of 20 and 40"
query = "What are the errors in my logs?"
#query = "Hown WudfCoInstaller are in log file?"

results = agent.run(query)
print(results)